In [1]:
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from scipy import stats
from fnn_datagenerator import BatchedFakeParticleDataset_All, flat_output_vars
from m1 import CustomKinematicNet, activation, hidden_layers, dfpath, modelsavepath, flat_output_vars, train_batch_size, prefix
from fnn_datagenerator import flat_output_vars, GeV_outputvars

import os
import sys
#os add to path /home/ddemler/HNLclassifier/DNN/
sys.path.append('/home/ddemler/HNLclassifier/DNN/')
from pytorch2python import create_data_loaders, create_test_loader
from tqdm.notebook import tqdm



14 112


In [2]:
data_basepath='/home/ddemler/HNLclassifier/saved_files/extracted_data/TEST10_'
# TEST10_data_newAug28
fulldatapath=data_basepath + 'train_Aug29'
valdatapath=data_basepath + 'val_Aug29'
testdatapath=data_basepath + 'test_Aug29'

fulldata=pd.read_pickle(fulldatapath)
valdata=pd.read_pickle(valdatapath)

In [3]:
print(fulldata.columns)

Index(['event', 'weightOriginal', 'charge_1', 'charge_2', 'charge_3', 'pt_1',
       'pt_2', 'pt_3', 'pt_MET', 'eta_1',
       ...
       'moth_pz_23', 'moth_E_12', 'moth_E_13', 'moth_E_23', 'E_tot',
       'mass_hyp', 'signal_label', 'channel', 'event_type', 'weightNorm'],
      dtype='object', length=137)


In [4]:

# old_model_input= ['1_eta', '1_mass', '1_phi', '1_pt', '2_eta?', '2_mass', '2_phi', '2_pt', '3_eta', '3_mass', '3_phi', '3_pt', 'MET_phi', 'MET_pt']
renamed_old_input_names=['eta_1', 'mass_1', 'phi_1', 'pt_1', 'eta_2', 'mass_2', 'phi_2', 'pt_2', 'eta_3', 'mass_3', 'phi_3', 'pt_3', 'phi_MET', 'pt_MET']


def datamaker(data):
    outputdata_shape= (len(data['pt_MET']), len(flat_output_vars))
    inputdata_shape= (len(data['pt_MET']), len(renamed_old_input_names))

    inputdata=np.empty(inputdata_shape)
    outputdata=np.empty(outputdata_shape)

    for i, outvar in enumerate(flat_output_vars):
        if outvar in GeV_outputvars:
            outputdata[:,i]=data[outvar]/data['E_tot']
        else:
            outputdata[:,i]=data[flat_output_vars[i]]

    for i in range(len(renamed_old_input_names)):
        inputdata[:,i]=data[renamed_old_input_names[i]]

    input_tensor = torch.tensor(inputdata, dtype=torch.float32)
    output_tensor = torch.tensor(outputdata, dtype=torch.float32)

    return input_tensor, output_tensor

train_inputdata, train_outputdata=datamaker(fulldata)
val_inputdata, val_outputdata=datamaker(valdata)


# outputdata_shape= (len(fulldata['pt_MET']), len(flat_output_vars))
# inputdata_shape= (len(fulldata['pt_MET']), len(renamed_old_input_names))

# inputdata=np.empty(inputdata_shape)
# outputdata=np.empty(outputdata_shape)

# for i in range(len(flat_output_vars)):
#     outputdata[:,i]=fulldata[flat_output_vars[i]]

# for i in range(len(renamed_old_input_names)):
#     inputdata[:,i]=fulldata[renamed_old_input_names[i]]

print("inputdata shape: ", train_inputdata.shape)
print("outputdata shape: ", train_outputdata.shape)
print("val inputdata shape: ", val_inputdata.shape)



inputdata shape:  torch.Size([124208, 14])
outputdata shape:  torch.Size([124208, 112])
val inputdata shape:  torch.Size([24841, 14])


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size=320
train_dataset = torch.utils.data.TensorDataset(train_inputdata, train_outputdata)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)


val_dataset = torch.utils.data.TensorDataset(val_inputdata, val_outputdata)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [6]:
model= CustomKinematicNet(input_size=14, hidden_layers=hidden_layers, lenoutput=len(flat_output_vars), activation_fn=activation)
model.load_state_dict(torch.load(modelsavepath))
model.to(device)

CustomKinematicNet(
  (layers): ModuleList(
    (0): Linear(in_features=14, out_features=512, bias=True)
    (1-2): 2 x Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=526, out_features=512, bias=True)
    (4-5): 2 x Linear(in_features=512, out_features=512, bias=True)
    (6): Linear(in_features=526, out_features=512, bias=True)
    (7-8): 2 x Linear(in_features=512, out_features=512, bias=True)
    (9): Linear(in_features=526, out_features=512, bias=True)
    (10-11): 2 x Linear(in_features=512, out_features=512, bias=True)
    (12): Linear(in_features=526, out_features=512, bias=True)
    (13-14): 2 x Linear(in_features=512, out_features=512, bias=True)
    (15): Linear(in_features=526, out_features=512, bias=True)
    (16-17): 2 x Linear(in_features=512, out_features=512, bias=True)
    (18): Linear(in_features=526, out_features=512, bias=True)
    (19): Linear(in_features=512, out_features=512, bias=True)
    (20): Linear(in_features=512, out_featu

In [7]:
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [10]:
out_feats=flat_output_vars
plt.style.use('ggplot')

residuals=np.array([])
y_total=np.array([])
y_pred_total=np.array([])
for i, (x,y) in enumerate(train_dataloader):
    x = x.to(device)
    y = y.to(device)
    y_pred = model(x)
    y_pred_total=np.append(y_pred_total, y_pred.cpu().detach().numpy())
    y_total=np.append(y_total, y.cpu().detach().numpy())


    # residuals=np.append(residuals, y_pred.cpu().detach().numpy() - y.cpu().detach().numpy())

    # residuals.append(y_pred.cpu().detach().numpy() - y.cpu().detach().numpy())


numfeatures=len(out_feats)
y_pred_total = y_pred_total.reshape(-1,numfeatures)
y_total = y_total.reshape(-1,numfeatures)

print(y_pred_total.shape)
print(y_total.shape)

residuals = [[] for _ in range(numfeatures)]
label_values = [[] for _ in range(numfeatures)]

# for i in range(numfeatures):
for i in tqdm(range(numfeatures), total=numfeatures, desc='Calculating residuals'):
    y_curr=y_total[:,i]
    # print("ycurr shape before reshape", y_curr.shape)
    
    # print("ycurr shape after reshape", y_curr.shape)
    y_pred_curr=y_pred_total[:,i]
    # y_pred_curr=y_pred_curr.reshape(-1,1)
    # print("ypredcurr shape after reshape", y_pred_curr.shape)
    residuals_curr = y_pred_curr - y_curr
    residuals[i]=residuals_curr
    label_values[i]=y_curr

residuals = [np.array(res_list) for res_list in residuals]  # Convert lists of arrays to arrays
# residual_medians = [np.median(res) for res in residuals]
residual_std_devs = [np.std(res) for res in residuals]
residual_means = [np.mean(res) for res in residuals]


num_rows = numfeatures // 3 + 1
num_cols = 3


fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(40, 250))
flat_axes = axes.flatten()

# for i, ax in enumerate(flat_axes[:numfeatures]):
for i, ax in tqdm(enumerate(flat_axes[:numfeatures]), total=numfeatures, desc='Plotting residuals'):
    ax.hist(residuals[i], bins=100, edgecolor='k', alpha=0.65)
    ax.axvline(x=residual_means[i] + residual_std_devs[i], color='r', linestyle='--', label=f'+1 std = {residual_means[i] + residual_std_devs[i]:.2f})')
    ax.axvline(x=residual_means[i] - residual_std_devs[i], color='b', linestyle='--', label=f'-1 std = {residual_means[i] - residual_std_devs[i]:.2f})')
    ax.set_title(f'Residuals for {out_feats[i]}')
    ax.set_yscale('log')
    ax.set_xlabel('Residual Value')
    ax.set_ylabel('Frequency')
    ax.legend()
    
    # Display the mean value on the plot
    mean_text = f"Mean: {residual_means[i]:.2f}, std: {residual_std_devs[i]:.5f}"
    ax.text(0.6, 0.85, mean_text, transform=ax.transAxes)
    

# for ax in flat_axes[numfeatures:]:
#     ax.axis('off')

plt.tight_layout()
plt.savefig(f'{prefix}_realdata_residuals.png')
plt.show()
    
    


(124208, 112)
(124208, 112)


Calculating residuals:   0%|          | 0/112 [00:00<?, ?it/s]

Plotting residuals:   0%|          | 0/112 [00:00<?, ?it/s]

/tmp/ddemler/ipykernel_74211/826537227.py:75: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  plt.tight_layout()
/tmp/ddemler/ipykernel_74211/826537227.py:76: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  plt.savefig(f'{prefix}_realdata_residuals.png')
/home/ddemler/.conda/envs/Dmitri-conda/lib/python3.8/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)


In [86]:
class DNN_flexible_with_kinematic(nn.Module):
    def __init__(self, kinematic_model, hidden_layer_sizes):
        super(DNN_flexible_with_kinematic, self).__init__()

        # Set kinematic_model parameters to not require gradients
        for param in kinematic_model.parameters():
            param.requires_grad = False
        
        self.kinematic_model = kinematic_model
        last_kinematic_output_size = kinematic_model.layers[-2].out_features
        
        layer_sizes = [last_kinematic_output_size] + hidden_layer_sizes + [1]
        
        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Get the output from kinematic_model (excluding the last layer)
        with torch.no_grad():
            for idx, layer in enumerate(self.kinematic_model.layers[:-1]):
                if (idx % 3 == 0) and (idx != 0):
                    x = self.kinematic_model.activation_fn(layer(torch.cat((x, x), dim=-1)))
                else:
                    x = self.kinematic_model.activation_fn(layer(x))
        
        # Pass the output through the classifier layers
        for i in range(len(self.layers) - 1):
            x = self.relu(self.layers[i](x))
        x = self.sigmoid(self.layers[-1](x))
        return x
